# Start a Simulated SNMP Agent

In [1]:
import subprocess

try:
    if process:
        pass
except:
    process = subprocess.Popen(
        [
            'snmpsimd.py',
            '--agent-udpv4-endpoint=127.0.0.1:1161',
            '--agent-udpv6-endpoint=[::1]:1161'
        ],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        shell=False
    )

# Start a Distributed Local Cluster

In [2]:
from dask import delayed
from distributed import Client, LocalCluster, as_completed

try:
    if cluster and client:
        pass
except:
    cluster = LocalCluster()
    client = Client(cluster)
    
client

Client Scheduler: tcp://127.0.0.1:38405,Cluster Workers: 4 Cores: 12 Memory: 67.39 GB


# Create a DataFrame Mimicking a Database Query Result

In [3]:
from uuid import UUID

import pandas as pd

df = (
    pd.DataFrame(
        data=[
            [UUID('3a8083c9-79ff-4ed9-969c-e204cee391b3'), 'host1', 'localhost:1161', 'recorded/linux-full-walk', 'DNS Resolution'],
            [UUID('83f73383-c7e6-44ff-a063-8c79f339777b'), 'host2', '127.0.0.1', 'recorded/linux-full-walk', 'Timeout'],
            [UUID('3cae774a-ec98-4679-b20a-dc1947b0546f'), 'host3', '[::1]:1161', 'recorded/linux-full-walk', 'IPv6'],
            [UUID('3cae774a-ec98-4679-b20a-dc1947b0546f'), 'host3', '[::1]:1161', 'recorded/linux-full-walk', 'Preserved Index'],
        ],
        columns=['id', 'hostname', 'ip_address', 'community_string', 'notes']
    )
    .set_index(['id', 'hostname'])
)
df

ip_address  \
id                                   hostname                   
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1     localhost:1161   
83f73383-c7e6-44ff-a063-8c79f339777b host2          127.0.0.1   
3cae774a-ec98-4679-b20a-dc1947b0546f host3         [::1]:1161   
                                     host3         [::1]:1161   

                                                       community_string  \
id                                   hostname                             
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1     recorded/linux-full-walk   
83f73383-c7e6-44ff-a063-8c79f339777b host2     recorded/linux-full-walk   
3cae774a-ec98-4679-b20a-dc1947b0546f host3     recorded/linux-full-walk   
                                     host3     recorded/linux-full-walk   

                                                         notes  
id                                   hostname                   
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1      DNS Resolution  
83f73383-c7e6-44ff-a063-8c79f339777b host2             Timeout  
3cae774a-ec98-4679-b20a-dc1947b0546f host3                IPv6  
                                     host3     Preserved Index

# Construct a MIB-like Definition of the SNMP Objects to Collect

In [4]:
import numpy as np

from snmp_fetch import ObjectType, object_type, pipeline_hook

@object_type(oid='.1.3.6.1.2.1')
class InterfaceTable(ObjectType):
    """A collection of SNMP interface tables that share the same index."""

    index=np.dtype([('if_index', np.uint64)])  # shared index collected after the full OID has been constructed
    
    @pipeline_hook('before_pivot')
    def set_index(df):
        return df.set_index('if_index')
    
    @pipeline_hook('after_merge')
    def set_index_type(df):
        df['if_index'] = df['if_index'].astype(pd.UInt64Dtype())
        return df

@object_type(parent=InterfaceTable, oid='.2.2.1')
class IfTable(ObjectType):
    """SNMPv2 IfTable."""
    
@object_type(parent=IfTable, oid='.7')
class IfAdminStatus(ObjectType):
    """IfTable.AdminStatus."""

    dtype=np.dtype([('admin_status', np.uint64)])
    
    @pipeline_hook('before_pivot')
    def set_type(df):
        df['admin_status'] = df['admin_status'].astype(pd.UInt64Dtype())
        return df
    
@object_type(parent=IfTable, oid='.8')
class IfOperStatus(ObjectType):
    """IfTable.OperStatus."""

    dtype=np.dtype([('oper_status', np.uint64)])
    
    @pipeline_hook('before_pivot')
    def set_type(df):
        df['oper_status'] = df['oper_status'].astype(pd.UInt64Dtype())
        return df

@object_type(parent=InterfaceTable, oid='.31.1.1.1')
class IfXTable(ObjectType):
    """SNMPV2 IfXTable"""

@object_type(parent=IfXTable, oid='.1')
class IfXAlias(ObjectType):
    """IfXTable Alias"""

    dtype=np.dtype([('alias', 'S256')])
    
    @pipeline_hook('before_pivot')
    def set_type(df):
        df['alias'] = df['alias'].str.decode('utf-8', errors='ignore')
        return df

InterfaceTable.describe()

InterfaceTable OBJECT-TYPE
    INDEX           {'if_index': (dtype('uint64'), 0)}
    DESCRIPTION
        A collection of SNMP interface tables that share the same index.
    ::= { .1.3.6.1.2.1 }

InterfaceTable ::= SEQUENCE {
    IfTable
    IfXTable
}

IfTable OBJECT-TYPE
    DESCRIPTION
        SNMPv2 IfTable.
    ::= { InterfaceTable .2.2.1 }

IfTable ::= SEQUENCE {
    IfAdminStatus
    IfOperStatus
}

IfAdminStatus OBJECT-TYPE
    BASE_TYPE       {'admin_status': (dtype('uint64'), 0)}
    DESCRIPTION
        IfTable.AdminStatus.
    ::= { IfTable .7 }

IfOperStatus OBJECT-TYPE
    BASE_TYPE       {'oper_status': (dtype('uint64'), 0)}
    DESCRIPTION
        IfTable.OperStatus.
    ::= { IfTable .8 }

IfXTable OBJECT-TYPE
    DESCRIPTION
        SNMPV2 IfXTable
    ::= { InterfaceTable .31.1.1.1 }

IfXAlias OBJECT-TYPE
    BASE_TYPE       {'alias': (dtype('S256'), 0)}
    DESCRIPTION
        IfXTable Alias
    ::= { IfXTable .1 }


In [5]:
from toolz.sandbox.core import unzip

from dask import delayed

from snmp_fetch import PduType, SnmpConfig
from snmp_fetch.distributed import distribute, fetch, to_pandas

graph = []

for hosts, data, index in distribute(
        df,
        batch_size=1,
        host='ip_address',
        snmp_community='community_string'
):
    response = delayed(fetch)(PduType.BULKGET, hosts, InterfaceTable, config=SnmpConfig(retries=0, timeout=1))
    result = delayed(to_pandas)(InterfaceTable, response, data, index)
    graph.append(result)

result_dfs, error_lists = unzip(client.gather(client.compute(graph)))

errors = [error for errors in error_lists for error in errors]
errors

[SnmpError(type=TIMEOUT_ERROR, Host(index=1, hostname='127.0.0.1', community='recorded/linux-full-walk'), sys_errno=None, snmp_errno=-24, err_stat=None, err_index=None, err_oid=None, message='Timeout error')]

In [6]:
results = pd.concat(result_dfs, sort=True)
results

#timestamp  \
id                                   hostname                             
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1    2019-11-12 16:33:27+00:00   
                                     host1    2019-11-12 16:33:27+00:00   
83f73383-c7e6-44ff-a063-8c79f339777b host2                          NaT   
3cae774a-ec98-4679-b20a-dc1947b0546f host3    2019-11-12 16:33:27+00:00   
                                     host3    2019-11-12 16:33:27+00:00   
                                     host3    2019-11-12 16:33:27+00:00   
                                     host3    2019-11-12 16:33:27+00:00   

                                               admin_status alias  \
id                                   hostname                       
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1                1    lo   
                                     host1                1  eth0   
83f73383-c7e6-44ff-a063-8c79f339777b host2              NaN   NaN   
3cae774a-ec98-4679-b20a-dc1947b0546f host3                1    lo   
                                     host3                1  eth0   
                                     host3                1    lo   
                                     host3                1  eth0   

                                                       community_string  \
id                                   hostname                             
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1     recorded/linux-full-walk   
                                     host1     recorded/linux-full-walk   
83f73383-c7e6-44ff-a063-8c79f339777b host2     recorded/linux-full-walk   
3cae774a-ec98-4679-b20a-dc1947b0546f host3     recorded/linux-full-walk   
                                     host3     recorded/linux-full-walk   
                                     host3     recorded/linux-full-walk   
                                     host3     recorded/linux-full-walk   

                                               if_index      ip_address  \
id                                   hostname                             
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1            1  localhost:1161   
                                     host1            2  localhost:1161   
83f73383-c7e6-44ff-a063-8c79f339777b host2          NaN       127.0.0.1   
3cae774a-ec98-4679-b20a-dc1947b0546f host3            1      [::1]:1161   
                                     host3            2      [::1]:1161   
                                     host3            1      [::1]:1161   
                                     host3            2      [::1]:1161   

                                                         notes  oper_status  
id                                   hostname                                
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1      DNS Resolution            1  
                                     host1      DNS Resolution            1  
83f73383-c7e6-44ff-a063-8c79f339777b host2             Timeout          NaN  
3cae774a-ec98-4679-b20a-dc1947b0546f host3                IPv6            1  
                                     host3                IPv6            1  
                                     host3     Preserved Index            1  
                                     host3     Preserved Index            1

# Stop the Simulated SNMP Agent

In [7]:
process.kill()
process.communicate()
del process